In [5]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from pyspark.sql import functions as F
from pyspark.sql.types import *
import pandas as pd
from pyspark.sql.window import Window
import time
from pyspark.sql import SparkSession

/tmp/ipykernel_33370/1510302545.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [6]:
spark = SparkSession.builder \
       .master("local[*]") \
       .appName("test") \
       .config("spark.driver.memory", "80g")\
       .config("spark.executor.memory", "50g")\
       .config("spark.dirver.maxResultSize", "50g")\
       .config('spark.local.dir', '/dados10t/datalake/raw/checkpoint_dir')\
       .getOrCreate()
sc = spark.sparkContext
sc.setCheckpointDir('/dados10t/datalake/raw/checkpoint_dir')

##### id_vigivac

In [7]:
id_vigivac = spark.read.parquet("/dados10t/datalake/standard/vigivac/base_analise/id_paciente_to_id_vigivac/id_vigivac_id_paciente_09072022.parquet/")

##### person_id

In [8]:
person_id = spark.read.parquet("/dados10t/datalake/standard/omop/tabelas_omop_cmd_5.4_v1/tb_person.parquet/")

# sg

In [9]:
#sg = spark.read.parquet("/dados10t/datalake/standard/vigivac/SG_uniao/dbnotifica_tb_sindrome_gripal_from_pandas_18032023_unido_2020a2023.parquet/")
#sg = spark.read.parquet("/dados10t/datalake/standard/vigivac/SG/sg_long_18032023_v9.parquet/")
#sg = spark.read.parquet("/dados10t/datalake/standard/vigivac/SG/sg_long_09072022_v9.parquet/")
sg = spark.read.parquet('/dados10t/datalake/standard/omop/data_with_ids/sg_long_09072022_v9-id_omop-v3.parquet/')

##### Checking missing dates

In [14]:
#
sg.count()

82896519

In [133]:
# porcentagem de missing dt_not
sg.filter(F.col("dt_datanotificacao").isNotNull() ).count()/sg.count()

0.9999959467538077

##### data inicio 

In [26]:
# porcentagem de missing com data_teste
# = 0.9999967191795461
# nenhuma mudança
sg.filter(F.col("dt_datanotificacao").isNotNull() ).count()/sg.count()

0.9999959467538077

In [10]:
sg = sg.withColumn("visit_start_date", F.coalesce(F.col("dt_datanotificacao"), F.col("dt_datateste"), F.col('dt_datainiciosintomas')  ) )

In [11]:
sg = sg.filter(F.col('visit_start_date').isNotNull())

##### data fim

In [11]:
# how many people with data encerramento
sg.filter(F.col("dt_dataencerramento").isNotNull() ).count()/sg.count()

0.4260621877627599

In [139]:
# 
sg.filter(F.col("visit_start_date") > F.col("visit_end_date") ).count()/sg.count()

0.009443329707687787

In [143]:
#
sg.filter(F.col("visit_start_date").isNotNull() ).count()/sg.count()

1.0

In [12]:
sg = sg.withColumn("visit_end_date", F.coalesce(F.col("dt_dataencerramento"), F.col("visit_start_date") )  )

In [13]:
# when start_date is after end_date, use start_date for both
sg = sg.withColumn("visit_end_date", 
              F.when(F.col("visit_start_date") > F.col("visit_end_date"),
                     F.col("visit_start_date")
                    ).otherwise(F.col("visit_end_date")))

In [15]:
#
sg.filter(F.col("visit_start_date") > F.col("visit_end_date") ).select('visit_start_date','visit_end_date').show()

+----------------+--------------+
|visit_start_date|visit_end_date|
+----------------+--------------+
|      2020-07-01|    2020-06-30|
|      2020-06-08|    2020-06-01|
|      2021-03-31|    2021-03-06|
|      2020-09-15|    2020-09-01|
|      2021-01-17|    2021-01-09|
|      2020-11-29|    2020-11-28|
|      2020-06-16|    2020-06-03|
|      2021-06-03|    2021-06-02|
|      2020-08-03|    2020-07-30|
|      2021-07-07|    2021-07-05|
|      2020-06-01|    2020-05-26|
|      2021-07-15|    2021-07-14|
|      2022-03-10|    2020-03-10|
|      2020-07-30|    2020-07-29|
|      2022-01-19|    2020-01-22|
|      2021-09-02|    2021-08-27|
|      2021-05-11|    2021-03-14|
|      2020-07-30|    2020-03-31|
|      2022-01-21|    2022-01-04|
|      2021-08-11|    2021-05-11|
+----------------+--------------+
only showing top 20 rows



##### check diff in days

In [10]:
#
sg_days = sg.withColumn('days_diff', F.datediff( F.col('visit_end_date'), F.col('visit_start_date')) )

In [19]:
#
sg_days.groupBy('days_diff').count().orderBy(F.col('count').desc()).show()

+---------+--------+
|days_diff|   count|
+---------+--------+
|        0|53092848|
|        1| 1393527|
|        7| 1243859|
|        6| 1142699|
|        2| 1139177|
|        3| 1136140|
|        4| 1088107|
|        5| 1060690|
|        8|  991219|
|       10|  892551|
|        9|  853085|
|       14|  810129|
|       11|  750813|
|       13|  732654|
|       12|  690018|
|       15|  621840|
|       16|  481528|
|       17|  428639|
|       20|  407029|
|       21|  396296|
+---------+--------+
only showing top 20 rows



##### visit_concept_id 

In [14]:
# sg = sg_days.withColumn('visit_concept_id', F.when(F.col('days_diff') == 0, 9202).otherwise(9201))
sg = sg.withColumn('visit_concept_id', F.lit(9202))

##### visit_type_concept_id 

In [15]:
# EHR type
sg = sg.withColumn('visit_type_concept_id', F.lit(32817))

##### care_site_id 

In [16]:
sg = sg.withColumn('care_site_id', F.lit(None))

##### person_id

In [12]:
#
sg.printSchema()

root
 |-- dt_datanotificacao: date (nullable = true)
 |-- dt_datainiciosintomas: date (nullable = true)
 |-- dt_datateste: date (nullable = true)
 |-- condicoes: string (nullable = true)
 |-- sintomas: string (nullable = true)
 |-- classificacaofinal: string (nullable = true)
 |-- dt_dataencerramento: date (nullable = true)
 |-- evolucaocaso: string (nullable = true)
 |-- tipoteste: string (nullable = true)
 |-- resultadoteste: string (nullable = true)
 |-- codigotipoteste: string (nullable = true)
 |-- codigoresultadoteste: string (nullable = true)
 |-- estadonotificacao: string (nullable = true)
 |-- municipio: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- idade: double (nullable = true)
 |-- codigo_ibge_resid: string (nullable = true)
 |-- profissionalsaude: string (nullable = true)
 |-- cbo: string (nullable = true)
 |-- evolucaocaso_2: string (nullable = true)
 |-- classificacaofinal_2: string (nullable = true)
 |-- sexo_2: string (nullable = true)
 |-- racac

In [23]:
sg.count()

82896502

In [17]:
sg_select = sg.select('id_vigvac', 'visit_occurrence_id', 'visit_concept_id', 'visit_start_date', 'visit_end_date', 'care_site_id', 'visit_type_concept_id')

# vacinacao 

In [18]:
# vac = spark.read.parquet('/dados10t/datalake/standard/vigivac/vacinacao/vacinacao07072022_pessoasunicas_5doses_v17.parquet/')
vac_long = spark.read.parquet('/dados10t/datalake/standard/omop/data_with_ids/vacinacao07072022_pessoasunicas_5doses_v17-id_omop-v3.parquet/')

In [29]:
vac_long.count()

446217353

In [19]:
vac_long = vac_long.withColumn('estabelecimento_valor', 
                    F.when(F.col('n_vacina')=='d1_data_aplicacao', F.col('d1_estab_valor'))\
                    .when(F.col('n_vacina')=='d2_data_aplicacao', F.col('d2_estab_valor'))\
                    .when(F.col('n_vacina')=='d3_data_aplicacao', F.col('d3_estab_valor'))\
                    .when(F.col('n_vacina')=='d4_data_aplicacao', F.col('d4_estab_valor'))\
                    .when(F.col('n_vacina')=='d5_data_aplicacao', F.col('d5_estab_valor'))
                   )

In [28]:
# vac_long = vac.selectExpr("id_vigvac",
#                          "stack(5,'d1_data_aplicacao',d1_data_aplicacao,'d2_data_aplicacao',d2_data_aplicacao,'d3_data_aplicacao',d3_data_aplicacao,'d4_data_aplicacao',d4_data_aplicacao,'d5_data_aplicacao',d5_data_aplicacao) as (variable,data_entrada)" ).filter(F.col("data_entrada").isNotNull())

In [161]:
# vac_total = vac_plus_raw.count()

##### start and end date

In [20]:
# start and end date for vaccination are the same
vac_long = vac_long.withColumn("visit_start_date", F.col('data_aplicacao'))
vac_long = vac_long.withColumn("visit_end_date", F.col('data_aplicacao'))

##### visit_concept_id

In [21]:
# pharmacy concept id
vac_long = vac_long.withColumn('visit_concept_id', F.lit(38004338))

##### visit_type_concept_id

In [22]:
# EHR concept id
vac_long = vac_long.withColumn('visit_type_concept_id', F.lit(32817))

##### care_site_id 

In [23]:
vac_long = vac_long.withColumn('care_site_id', F.col('estabelecimento_valor'))

##### person_id

In [24]:
vac_select = vac_long.select('id_vigvac', 'visit_occurrence_id', 'visit_concept_id', 'visit_start_date', 'visit_end_date', 'care_site_id', 'visit_type_concept_id')

In [36]:
#
vac_select.count()

446217353

# srag 

In [25]:
# srag = spark.read.parquet('/dados10t/datalake/standard/vigivac/srag/dbsrag_tb_influd_long_28032023_v8.parquet')
# srag = spark.read.parquet("/dados10t/datalake/standard/vigivac/srag/dbsrag_tb_influd_long_09072022_v8.parquet/")
srag = spark.read.parquet('/dados10t/datalake/standard/omop/data_with_ids/dbsrag_tb_influd_long_09072022_v8-id_omop-v3.parquet/')

In [38]:
#
srag.count()

2487085

##### start date

In [39]:
#
srag.filter(F.col('dt_interna').isNotNull()).count()/srag.count()

0.9350762036681497

In [90]:
#
srag.filter(F.col('dt_notif_dt').isNotNull()).count()/srag.count()

1.0

In [27]:
srag.printSchema()

root
 |-- id_paciente: string (nullable = true)
 |-- st_notific: string (nullable = true)
 |-- dt_notific: string (nullable = true)
 |-- sem_not: string (nullable = true)
 |-- dt_sin_pri: string (nullable = true)
 |-- sem_pri: string (nullable = true)
 |-- sg_uf_not: string (nullable = true)
 |-- id_regiona: string (nullable = true)
 |-- co_regiona: decimal(10,0) (nullable = true)
 |-- id_municip: string (nullable = true)
 |-- co_mun_not: string (nullable = true)
 |-- id_unidade: string (nullable = true)
 |-- co_uni_not: string (nullable = true)
 |-- cs_sexo: string (nullable = true)
 |-- dt_nasc: string (nullable = true)
 |-- nu_idade_n: decimal(10,0) (nullable = true)
 |-- tp_idade: string (nullable = true)
 |-- cod_idade: string (nullable = true)
 |-- cs_gestant: decimal(10,0) (nullable = true)
 |-- cs_raca: string (nullable = true)
 |-- cs_etinia: string (nullable = true)
 |-- cs_escol_n: decimal(10,0) (nullable = true)
 |-- nu_cep: string (nullable = true)
 |-- id_pais: string (nu

In [29]:
srag = srag.withColumn('visit_start_date', 
                       F.coalesce(
                           F.to_date(F.col('dt_interna'), 'dd/MM/yyyy'), 
                           F.to_date(F.col('dt_notific'), 'dd/MM/yyyy')
                       )
                      )

##### end date

In [41]:
#
srag.filter(F.col('dt_encerra_dt').isNotNull()).count()/srag.count()

0.8969994994139726

In [42]:
#
srag.filter(F.col('dt_evoluca').isNotNull()).count()/srag.count()

0.831092222421027

In [30]:
srag = srag.withColumn('visit_end_date', 
                       F.coalesce(
                           F.to_date(F.col('dt_encerra'), 'dd/MM/yyyy'), 
                           F.to_date(F.col('dt_evoluca'), 'dd/MM/yyyy'), 
                           F.col('visit_start_date')
                       ) )

##### checking quality

In [91]:
srag.filter(F.col('visit_start_date') > F.col('visit_end_date')).count()/srag.count()

0.0008777343757853069

In [24]:
#
srag_days = srag.withColumn('days_diff', F.datediff( F.col('visit_end_date'), F.col('visit_start_date')) )

In [29]:
#
srag_days.groupBy('days_diff').count().orderBy(F.col('count').desc()).show()

+---------+------+
|days_diff| count|
+---------+------+
|        0|277563|
|        7|100629|
|        6|100262|
|        5| 97075|
|        4| 91595|
|        8| 91395|
|        9| 82295|
|        3| 80601|
|       10| 76533|
|       11| 69192|
|       12| 63802|
|        2| 62186|
|       13| 60240|
|       14| 57219|
|        1| 51815|
|       15| 51453|
|       16| 46263|
|       17| 42893|
|       18| 39667|
|       19| 36782|
+---------+------+
only showing top 20 rows



##### visit_concept_id

In [31]:
srag = srag.withColumn('visit_concept_id', F.lit(9201))
# srag = srag_days.withColumn('visit_concept_id', F.when(F.col('days_diff') == 0, 9202).otherwiseb(9201))

##### visit_type_concept_id 

In [32]:
srag = srag.withColumn('visit_type_concept_id', F.lit(32817))

##### care_site_id 

In [46]:
#
srag.select('co_uni_not').show()

+----------+
|co_uni_not|
+----------+
|   0023051|
|   6668143|
|   2134276|
|   8000956|
|   2249472|
|   2527057|
|   7257406|
|   7866801|
|   2615797|
|   2078473|
|   2263823|
|   2576341|
|   7107366|
|   0012521|
|   9383298|
|   2089777|
|   2352516|
|   2022877|
|   2200902|
|   2079127|
+----------+
only showing top 20 rows



In [33]:
srag = srag.withColumn('care_site_id', F.col('co_uni_not'))

##### person_id

In [35]:
srag.count()

2490196

In [36]:
srag_vigvac = srag.join(id_vigivac, ['id_paciente'], 'inner')

In [37]:
srag_vigvac.count()

2490196

In [38]:
srag_select = srag_vigvac.select('id_vigvac', 'visit_occurrence_id', 'visit_concept_id', 'visit_start_date', 'visit_end_date', 'care_site_id', 'visit_type_concept_id')

In [50]:
srag_select.count()

2487085

# Union

In [39]:
srag_sg = srag_select.union(sg_select)

In [40]:
srag_sg_vac = srag_sg.union(vac_select)

In [41]:
srag_sg_vac = srag_sg_vac.checkpoint()

In [37]:
srag_sg_vac.columns

['id_vigvac',
 'visit_occurrence_id',
 'visit_concept_id',
 'visit_start_date',
 'visit_end_date',
 'care_site_id',
 'visit_type_concept_id']

In [38]:
srag_sg_vac.count()

531600940

# Add person_id

In [44]:
person_id = person_id.withColumnRenamed('id_vigivac', 'id_vigvac')

In [187]:
#
person_id.columns

['person_id',
 'gender_source_value',
 'race_source_value',
 'gender_concept_id',
 'race_concept_id',
 'year_of_birth',
 'month_of_birth',
 'day_of_birth',
 'birth_datetime',
 'ethnicity_concept_id',
 'id_vigvac']

In [179]:
#
srag_sg_vac.columns

['id_vigvac',
 'visit_concept_id',
 'visit_start_date',
 'visit_end_date',
 'care_site_id',
 'visit_type_concept_id']

In [45]:
full_id = srag_sg_vac.join(person_id, 'id_vigvac', 'inner').select(srag_sg_vac.columns + ['person_id'])

In [189]:
full_id.columns

['id_vigvac',
 'visit_concept_id',
 'visit_start_date',
 'visit_end_date',
 'care_site_id',
 'visit_type_concept_id',
 'person_id']

In [39]:
full_id.count()

530582227

## remove care_site_id numbers that aren't present in cnae

In [46]:
cnae = spark.read.parquet("/dados10t/datalake/standard/omop/tabelas_omop_cmd_5.4_v1/tb_care_site.parquet")

In [47]:
table_p_cnae = full_id.join(cnae, 'care_site_id' ,'left')

In [48]:
table_p_cnae = table_p_cnae.withColumn('valid_cnae',
                                       F.when(F.col('care_site_source_value').isNull() & F.col('care_site_id').isNotNull(), 0)
                                       .otherwise(1))

In [49]:
table_p_cnae = table_p_cnae.withColumn('care_site_id',
                                       F.when(F.col('valid_cnae') == 0, None)\
                                       .otherwise(F.col('care_site_id'))
                                      )

In [50]:
table_p_cnae = table_p_cnae.select(full_id.columns)

### saving dataset

In [51]:
table_p_cnae\
.drop('id_vigvac')\
.write.parquet("/dados10t/datalake/standard/omop/tabelas_omop_cmd_5.4_v1/tb_visit_occurrence_data.parquet")